In [1]:
# imports
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
# function to load the datasets
def load_ds(set_name, train_split, validation_split):
    train_data, validation_data, test_data = tfds.load(name=set_name, split=('train[:' + str(train_split) + '%]', 'train[' + str(validation_split) + '%:]', 'test'), as_supervised=True)
    return train_data, validation_data, test_data

In [4]:
# load data
train_data, validation_data, test_data = load_ds('imdb_reviews', 60, 60)

Shuffling and writing examples to C:\Users\lakha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete8VJSTD\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\lakha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete8VJSTD\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\lakha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete8VJSTD\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\lakha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [6]:
# compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train the model
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 213ms/step - loss: 0.6527 - accuracy: 0.6133 - val_loss: 0.6212 - val_accuracy: 0.6333
Epoch 2/20
30/30 [==============================] - 4s 145ms/step - loss: 0.6002 - accuracy: 0.6582 - val_loss: 0.5811 - val_accuracy: 0.6686
Epoch 3/20
30/30 [==============================] - 5s 176ms/step - loss: 0.5590 - accuracy: 0.6891 - val_loss: 0.5464 - val_accuracy: 0.6994
Epoch 4/20
30/30 [==============================] - 5s 157ms/step - loss: 0.5206 - accuracy: 0.7250 - val_loss: 0.5141 - val_accuracy: 0.7260
Epoch 5/20
30/30 [==============================] - 4s 128ms/step - loss: 0.4838 - accuracy: 0.7541 - val_loss: 0.4851 - val_accuracy: 0.7573
Epoch 6/20
30/30 [==============================] - 5s 159ms/step - loss: 0.4481 - accuracy: 0.7775 - val_loss: 0.4545 - val_accuracy: 0.7729
Epoch 7/20
30/30 [==============================] - 5s 156ms/step - loss: 0.4124 - accuracy: 0.8008 - val_loss: 0.4294 - val_accuracy: 0.7977
Epoch 

In [7]:
# evaluate
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 6s - loss: 0.3291 - accuracy: 0.8578
loss: 0.329
accuracy: 0.858
